In [46]:
from utils.postgre_database_connection import connect_to_local_database

import psycopg2
import pandas as pd
import glob
import random
import string

In [2]:
iris_path = glob.glob('*/IRIS.csv')[0]
iris_df = pd.read_csv(iris_path)

In [3]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
columns_name = iris_df.columns
features = iris_df.drop(columns='species')
target = iris_df['species']

In [5]:
features[:5]

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
target[:5]

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: species, dtype: object

In [7]:
columns_name

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

### 1. Membuat Koneksi ke PostgreSQL

In [42]:
try: 
    connection = connect_to_local_database()
    cursor = connection.cursor()
    print("Successfully connected to the database.")
    
    # cek konfigurasi apakah sudah sesuai atau belum, kalo ragu bisa di uncomment
    # cursor.close()
    # connection.close() 
except (Exception, psycopg2.DatabaseError) as error:
    print(f'Error: {error}')

Successfully connected to the database.


### Membuat tabel iris di database

In [44]:
# Membuat tabel untuk menyimpan data iris
create_table_query = """
--sql
create table public.barang(
    kode_barang varchar(50) not null,
    nama_barang varchar(100) not null,
    harga_barang float not null default 0.0,
    jumlah_barang int not null default 0,
    waktu_barang_tercetak timestamp not null default current_timestamp
)
;
"""

cursor.execute(create_table_query)
connection.commit()
print('Tabel barang berhasil dibuat')
cursor.close()
connection.close() 

Tabel barang berhasil dibuat


In [47]:
# Fungsi untuk menghasilkan kode barang acak
def generate_kode_barang():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Fungsi untuk menghasilkan nama barang acak
def generate_nama_barang():
    return 'Barang ' + ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))


In [ ]:
kode_barang = generate_kode_barang()
nama_barang = generate_nama_barang()
harga_barang = round(random.uniform(1.0, 1000.0), 2)
jumlah_barang = random.randint(1, 100)

In [ ]:
# Membuat koneksi ke database PostgreSQL
try:
    connection = psycopg2.connect(
        host='localhost',
        database='angga_mlops_db',
        user='postgres',
        password='postgres'
    )
    cursor = connection.cursor()
    
    # Insert data iris ke tabel 'iris'
    insert_query = """
    --sql
        INSERT INTO iris (sepal_length, sepal_width, petal_length, petal_width, species)
        VALUES (%s, %s, %s, %s, %s);
    ;
    """
    
    for value in iris_df.values:
        cursor.execute(insert_query, tuple(value))
    
    connection.commit()
    print("Data berhasil diinsert ke tabel 'iris'.")
    
except (Exception, psycopg2.DatabaseError) as error:
    print(f"Error: {error}")
finally:
    if connection:
        cursor.close()
        connection.close()


Data berhasil diinsert ke tabel 'iris'.


In [ ]:
# Membuat koneksi ke database PostgreSQL
try:
    connection = psycopg2.connect(
        host='localhost',
        database='angga_mlops_db',
        user='postgres',
        password='postgres'
    )
    cursor = connection.cursor()
    
    # Mengambil data dari tabel 'iris'
    select_query = "SELECT * FROM iris;"
    cursor.execute(select_query)
    rows = cursor.fetchall()
    
    # Mengkonversi hasil query menjadi DataFrame
    df = pd.DataFrame(rows, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target'])
    print(df.head())  # Menampilkan 5 baris pertama dari DataFrame
    
except (Exception, psycopg2.DatabaseError) as error:
    print(f"Error: {error}")
finally:
    if connection:
        cursor.close()
        connection.close()

  sepal_length sepal_width petal_length petal_width       target
0          5.1         3.5          1.4         0.2  Iris-setosa
1          4.9         3.0          1.4         0.2  Iris-setosa
2          4.7         3.2          1.3         0.2  Iris-setosa
3          4.6         3.1          1.5         0.2  Iris-setosa
4          5.0         3.6          1.4         0.2  Iris-setosa


In [ ]:
df

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
